# Duck Hunting


<div class="alert alert-block alert-success">
<b>Resumen:</b> El código desarrolla un método para detectar y localizar un objeto específico dentro de una imagen mediante la técnica de correlación cruzada entre una plantilla (objeto) y una imagen de fondo.
</div>

***

## Contenido

1. Preámbulo
2. Lectura del fondo
3. Lectura del personaje
4. Correlación entre el personaje y el fondo
5. Fusión de las mediciones de correlación
6. Mapa de correlación
    - 6.1 Mapa de corelación 2D
    - 6.2 Mapa de corelació 3D
7. Coordenadas de la máxima correlación
8. Visualización del resultado
9. Preguntas orientadoras
10. Actividades

---

## 1. Preámbulo

Importamos las librerías necesarias:

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

## 2. Lectura de la imagen de fondo
Leemos la imagen de fondo y se muestra.

In [ ]:
# Lectura de la imagen de fondo
im_bg = cv2.imread("duck_hunt_1.png")
im_bg_float = im_bg.astype(np.float32)/255.0

# Convertimos de BGR a RGB para mostrar correctamente con matplotlib
plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(im_bg, cv2.COLOR_BGR2RGB))
plt.title("Imagen de fondo")
plt.axis('off')
plt.show()

# 3. Lectura del personaje
Leemos la imagen del personaje (pato).

In [ ]:
# Lectura de la imagen del personaje
im_duck = cv2.imread("duck_1.png")
im_duck_float = im_duck.astype(np.float32)/255.0

# Convertimos de BGR a RGB para mostrar correctamente con matplotlib
plt.figure(figsize=(3, 2))
plt.imshow(cv2.cvtColor(im_duck, cv2.COLOR_BGR2RGB))
plt.title("Imagen del personaje")
plt.axis('off')
plt.show()

## 4. Correlación entre el personaje y el fondo
Realizamos la correlación entre las imágenes en cada canal de color:

In [ ]:
# Correlación en el canal Rojo
duck_corr_r = cv2.filter2D(im_bg_float[:, :, 2], -1, im_duck_float[:, :, 2], borderType=cv2.BORDER_CONSTANT)

# Correlación en el canal Verde
duck_corr_g = cv2.filter2D(im_bg_float[:, :, 1], -1, im_duck_float[:, :, 1], borderType=cv2.BORDER_CONSTANT)

# Correlación en el canal Azul
duck_corr_b = cv2.filter2D(im_bg_float[:, :, 0], -1, im_duck_float[:, :, 0], borderType=cv2.BORDER_CONSTANT)

## 5. Fusión de las mediciones de correlación
Fusionamos las mediciones de correlación de los tres canales.

In [ ]:
duck_corr_rgb = np.sqrt(duck_corr_r**2 + duck_corr_g**2 + duck_corr_b**2)
print("Valor máximo de correlación:", np.max(duck_corr_rgb))
print("Valor minimo de correlación:", np.min(duck_corr_rgb))

## 6. Mapa de correlación

### 6.1 Normalización del mapa de correlación

Antes de mostrar el mapa de correlación, lo normalizamos para que los valores estén en el rango [0, 1].

In [ ]:
# Normalizamos el mapa de correlación
min_corr = np.min(duck_corr_rgb)
max_corr = np.max(duck_corr_rgb)
duck_corr_rgb_norm = (duck_corr_rgb - min_corr) / (max_corr - min_corr)

### 6.2 Mapa de correlación en 2D

Mostramos el mapa de correlación invertido en 2D:

In [ ]:
plt.imshow(duck_corr_rgb_norm, cmap='gray')
plt.title("Mapa de correlación (2D)")
plt.colorbar()
plt.show()

### 6.3 Mapa de correlación 3D

Mostramos el mapa de correlación 3D, invertimos la imagen para ajustarla con los ejes del plano 3D.

In [ ]:
# Invertimos verticalmente el mapa de correlación normalizado
duck_corr_rgb_norm_flipped = np.flipud(duck_corr_rgb_norm)

# Submuestreamos para reducir la cantidad de datos
duck_corr_norm_ds = duck_corr_rgb_norm_flipped[::2, ::2]

# Creamos una malla de coordenadas
X, Y = np.meshgrid(np.arange(duck_corr_norm_ds.shape[1]), np.arange(duck_corr_norm_ds.shape[0]))

# Graficamos la superficie
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, duck_corr_norm_ds, cmap='viridis')
ax.set_title("Mapa de correlación normalizado (3D)")
# Rotamos la figura 3D
ax.view_init(elev=25, azim=-85)  # Puedes ajustar 'elev' y 'azim' para cambiar la vista
plt.show()

## 7. Coordenadas de la máxima correlación

Encontramos las coordenadas donde la correlación es máxima.

In [ ]:
max_corr = np.max(duck_corr_r)
indices = np.where(duck_corr_r == max_corr)
r = indices[0][0]
c = indices[1][0]
print(f"Coordenadas de máxima correlación: r = {r}, c = {c}")

## 8. Visualización del resultado

Utilizamos la función cv2.rectangle de OpenCV para dibujar un recuadro en la posición encontrada, utilizando el tamaño de la imagen del pato.

In [ ]:
# Creamos una copia de la imagen de fondo para dibujar el resultado
im_bg_result = im_bg.copy()

# Obtenemos el tamaño de la imagen del pato
duck_height, duck_width = im_duck.shape[:2]

# Coordenadas del rectángulo
top_left = (c - duck_width//2, r - duck_height//2)
bottom_right = (c + duck_width//2, r + duck_height//2)

# Nos aseguramos de que las coordenadas estén dentro de los límites de la imagen
top_left = (max(top_left[0], 0), max(top_left[1], 0))
bottom_right = (min(bottom_right[0], im_bg_result.shape[1]-1), min(bottom_right[1], im_bg_result.shape[0]-1))

# Dibujamos el rectángulo en la imagen
cv2.rectangle(im_bg_result, top_left, bottom_right, color=(0, 0, 255), thickness=2)  # Color rojo en BGR

# Mostramos la imagen con la posición marcada
plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(im_bg_result, cv2.COLOR_BGR2RGB))
plt.title("Resultado final")
plt.axis('off')
plt.show()

## 9. Preguntas orientadoras

<b>Mejora de la precisión:</b>

- ¿De qué manera podemos ajustar la regla de fusión para mejorar la detección del objeto?
- ¿Qué sucede si cambiamos los pesos asignados a los canales de color en la fusión de las correlaciones?

<b>Escalabilidad y robustez:</b>

- ¿Cómo podemos adaptar el código para detectar objetos de diferentes tamaños o rotaciones?
- ¿Es posible implementar una técnica de correlación multiescala o rotacionalmente invariante?

<b>Preprocesamiento de imágenes:</b>

- ¿Qué técnicas de preprocesamiento (como filtrado, ecualización de histograma, normalización) podrían mejorar la calidad de la correlación?
- ¿Deberíamos considerar la conversión a otros espacios de color (como HSV o Lab) para mejorar la detección?

<b>Gestión de ruido y artefactos:</b>

- ¿Cómo afecta el ruido en las imágenes a la correlación y cómo podemos mitigarlo?
- ¿Podemos aplicar filtros pasa-bajos o técnicas de reducción de ruido antes de la correlación?

<b>Automatización y generalización:</b>

- ¿Cómo podemos automatizar la detección para múltiples objetos o patrones diferentes en la misma imagen?
- ¿Es posible entrenar un modelo que aprenda características más generales del objeto para mejorar la detección?

---

## 10. Actividades

<b>Ajustar la regla de fusión:</b>

- Actividad: Experimentar con diferentes pesos en la fusión de las correlaciones de los canales de color para observar cómo afecta a la detección del objeto.
- Modificación: Modificar los coeficientes en la ecuación de fusión.

<b>Cambiar el objeto de detección:</b>

- Actividad: Sustituir la imagen del pato por otra imagen de un objeto diferente (por ejemplo, el otro pato, etc.) y ajustar el código para detectar este nuevo objeto en la imagen de fondo.
- Modificación: Actualizar las rutas de las imágenes y, si es necesario, ajustar los pesos en la regla de fusión para adaptarse al nuevo objeto.

<b>Implementar una Región de Interés (ROI):</b>

- Actividad: Limitar el área de búsqueda a una región específica de la imagen de fondo para mejorar la eficiencia y la precisión de la detección.
- Modificación: Definir coordenadas para la ROI y aplicar la correlación únicamente dentro de esa región.